In [1]:
%%bash
# Download model
#wget https://www.cntk.ai/Models/CNTK_Pretrained/ResNet50_ImageNet_CNTK.model

In [2]:
import os
import sys
import numpy as np
import cntk as C
from cntk import load_model, combine
from common.params_inf import *
from common.utils import *

In [3]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [5]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [6]:
# Features (penultimate layer)
node_name = "z.x"
model_file = "ResNet50_ImageNet_CNTK.model"

In [7]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb_X(data, batchsize):
        pred = classifier.eval(dta)
        out[idx*batchsize:(idx+1)*batchsize] = pred[0].squeeze()
    return out

In [8]:
# Load model
loaded_model  = load_model(model_file)
node_in_graph = loaded_model.find_by_name(node_name)
output_nodes  = combine([node_in_graph.owner])

In [9]:
fake_input_data_cf = np.ascontiguousarray(fake_input_data_cf)
cold_start = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

In [10]:
%%time
features = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

CPU times: user 1.53 s, sys: 501 ms, total: 2.03 s
Wall time: 2.26 s


In [11]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/2.05))

Images per second 624.3902439024391
